In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
import base64
import requests
import hashlib

# 1. rsa pubkey 요청하기

In [2]:
pubkey_resp = requests.get(
    'http://127.0.0.1:8000/otpass/pubkey/'
)

In [3]:
pubkey = pubkey_resp.text

# 2. 키 객체 생성하기

In [4]:
pubkey_inst = serialization.load_pem_public_key(
    pubkey.encode('utf-8'),
    backend=default_backend()
)

# 3. 요청데이터 준비하고 패스워드만 rsa암호화 --> base64인코딩 (해시화는 넘어가서 한다)

In [5]:
url = 'http://127.0.0.1:8000/otpass/otpass_mail/'
request_data = {
    'email': "mafal201116@gmail.com",
    'pwd': 'Fmal1029!',
    'mail_idx': '',
}

# 암호화된 원본
# print(hashlib.sha512(request_data['pwd'].encode()).hexdigest())
# [아래] pwd를 바이너리로 변환 후, sha512적용 후 utf-8로 인코딩
# request_data['pwd'] = hashlib.sha512(request_data['pwd'].encode()).hexdigest().encode('utf-8') # sha516으로 전환
request_data['pwd'] = request_data['pwd'].encode('utf-8')
request_data['pwd'] = pubkey_inst.encrypt(
    request_data['pwd'],
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
request_data['pwd'] = base64.b64encode(request_data['pwd'])

request_header = {
    'User-Agent': 'dgnit-version231018'
}
request_data

{'email': 'mafal201116@gmail.com',
 'pwd': b'PpRhjAqGvUTNhjVrnu0i7AU/K4a0w3sEjFy66bucQh3+DaJVygwxpRpxroWg0n1DkHBsoAl4DRn8H4W6VAshB5VSTpC94Xt9WACg2VwTIQRiO96jnmD4rfHcx5f7e6qFIMEh6x6QK0bWvIGemVRStVkT7rqltfEfb4FkYVYl61PuLg3a/SFSJQwdZHwC/SF4jcXYxj4eOmbdVXcCsrpYeb3EutLZZ/Hes6R2Pj7vQOAKsxFP2gpizxI03QXzcdJI6vB9Ya0zqllrLlV59wbT3nQR14v28aova97c5BN32/Qcjcw8XwbKX5AojvEqgGAKwDbCemdDNEsIx7bBk1N5Ww==',
 'mail_idx': ''}

# 4. 요청하기(otp번호 요청 전/후 필요)
- 실제로 otp가 변하는지 확인을 하기 위해서
- otp발송이 완료되었다는 이벤트 확인 후 제한시간동안 주기적으로 확인
- 실제 반영까지 시간이 좀 걸림(좋은 환경에서 대략 10초 정도?)

In [9]:
# POST 요청 보내기
response = requests.post(url, 
                         data=request_data, 
                         headers=request_header)
response.text

'{"result": "SUCCESS", "answer": "타티나타애아\\r\\n"}'

# 5. OTP 있는지 확인하고 파씽하기
- 서비스별로 다름